In [28]:
# https://github.com/daveebbelaar/airbnb-whatsapp-bot/blob/main/app/services/openai_service.py
from openai import OpenAI
import shelve
# from dotenv import load_dotenv
import os
import time
import logging
logging.basicConfig(format='%(asctime)s %(message)s')

# load_dotenv()
client = OpenAI()

In [29]:

def upload_file(path):
    # Upload a file with an "assistants" purpose
    file = client.files.create(
        file=open("./data/airbnb-faq.pdf", "rb"), purpose="assistants"
    )
    return file


def create_assistant(file):
    """
    You currently cannot set the temperature for Assistant via the API.
    """
    assistant = client.beta.assistants.create(
        name="WhatsApp AirBnb Assistant",
        instructions="You're a helpful WhatsApp assistant that can assist guests that are staying in our Paris AirBnb. Use your knowledge base to best respond to customer queries. If you don't know the answer, say simply that you cannot help with question and advice to contact the host directly. Be friendly and funny.",
        tools=[{"type": "retrieval"}],
        model="gpt-4-1106-preview",
        file_ids=[file.id],
    )
    return assistant


# Use context manager to ensure the shelf file is closed properly
def check_if_thread_exists(wa_id):
    with shelve.open("threads_db") as threads_shelf:
        return threads_shelf.get(wa_id, None)


def store_thread(wa_id, thread_id):
    with shelve.open("threads_db", writeback=True) as threads_shelf:
        threads_shelf[wa_id] = thread_id


def run_assistant(thread, name):
    # Retrieve the Assistant
    assistant = client.beta.assistants.retrieve("asst_4bRlDD4TMebiulfR0vh1gejO")

    # Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        # instructions=f"You are having a conversation with {name}",
    )

    # Wait for completion
    # https://platform.openai.com/docs/assistants/how-it-works/runs-and-run-steps#:~:text=under%20failed_at.-,Polling%20for%20updates,-In%20order%20to
    while run.status != "completed":
        # Be nice to the API
        time.sleep(0.5)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Retrieve the Messages
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    new_message = messages.data[0].content[0].text.value
    logging.info(f"Generated message: {new_message}")
    return new_message


def generate_response(message_body, wa_id, name):
    # Check if there is already a thread_id for the wa_id
    thread_id = check_if_thread_exists(wa_id)

    # If a thread doesn't exist, create one and store it
    if thread_id is None:
        logging.info(f"Creating new thread for {name} with wa_id {wa_id}")
        thread = client.beta.threads.create()
        store_thread(wa_id, thread.id)
        thread_id = thread.id

    # Otherwise, retrieve the existing thread
    else:
        logging.info(f"Retrieving existing thread for {name} with wa_id {wa_id}")
        thread = client.beta.threads.retrieve(thread_id)

    # Add message to thread
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message_body,
    )

    # Run the assistant and get the new message
    new_message = run_assistant(thread, name)

    return new_message

In [18]:
file = upload_file(path = "")

In [20]:
assistant = create_assistant(file)

In [32]:
message_body = "what is the checkin time?"
generate_response(message_body, wa_id = "123", name = "Ting")

"The check-in time is after 3 PM. Don't forget that patience is a virtue, especially when it's rewarded with a cozy Parisian haven!【8†source】"

In [27]:
message_body = "Are there any local restaurants you'd suggest?"
generate_response(message_body, wa_id = "456", name = "Huanxin")

'Absolutely, I have some "à la carte" suggestions for your Parisian culinary adventure:\n\n- **La Petite Parisienne**, nestled at 15 Rue de Montmorency, is a real treat.\n- **Chez Julien**, conveniently located at 1 Rue du Pont Louis-Philippe, offers a taste of local flair.\n- And for a dash of Parisian charm, **Le Saint Régis** on 6 Rue Jean du Bellay awaits your visit.\n\nThey all offer delicious Parisian cuisine that can make your taste buds dance the can-can! Enjoy your gourmet journey! 🍽️✨'

In [45]:
thread_id = check_if_thread_exists(wa_id = "123")
thread_id

'thread_O9IHW6LdsMgtcbhQPZZL80NB'

In [35]:
thread_id = check_if_thread_exists(wa_id = "456")
thread_id

'thread_idOC1YaviCnRy3CSPeROWlla'

In [46]:
message_body = "Are there any local restaurants you'd suggest?"

In [47]:
# Add message to thread
message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message_body,
)

In [48]:
run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant.id,
        # instructions=f"You are having a conversation with {name}",
    )


In [52]:
run.status

'queued'

In [50]:
messages = client.beta.threads.messages.list(thread_id=thread_id)
new_message = messages.data[0].content[0].text.value

In [51]:
new_message

'Certainly! For a taste of Paris that will make your taste buds do a little happy dance, try these fantastic spots:\n\n- **La Petite Parisienne**, at 15 Rue de Montmorency\n- **Chez Julien**, at 1 Rue du Pont Louis-Philippe\n- **Le Saint Régis**, at 6 Rue Jean du Bellay\n\nThey are all highly recommended for their Parisian cuisine. Enjoy your gastronomic adventure!【13†source】'